Importing Pandas library

In [1]:
import pandas as pd

!pip install folium

import folium

Reading the data with pd.read_html

In [ ]:
!pip install folium
import folium

In [2]:
url = ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
table = pd.read_html(url)
table[0].head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Removing all rows that have Not assigned values for Borough

In [3]:
df = table[0]
df = df[df.Borough != 'Not assigned']

Changing any Not assigned values in Neighborhood to the corresponding Borough value. Also I am not sure why the notification popped up below the cell. It wasnt there before, when I run it again, its gone. Any input on that would be greatly appreciated.

In [15]:
df.loc[df['Neighbourhood'] == ('Not assigned'), 'Neighbourhood'] = df['Borough']


Grouping Neighborhood values by Postalcode

In [5]:

df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list)
df = df.sample(frac=1).reset_index()
df['Neighbourhood'] = df['Neighbourhood'].str.join(',')
df.head()

,Postcode,Borough,Neighbourhood
0,M1X,Scarborough,Upper Rouge
1,M9W,Etobicoke,Northwest
2,M3N,North York,Downsview Northwest
3,M4B,East York,"Woodbine Gardens,Parkview Hill"
4,M1R,Scarborough,"Maryvale,Wexford"


Checking the shape.

In [6]:
df.shape

(103, 3)

Getting the latitude and longitude data

In [7]:
url_geo = ('https://cocl.us/Geospatial_data')
df_geo = pd.read_csv(url_geo, header=0)

Taking a quick look at table_geo

In [8]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging df and df_geo and taking a quick look.

In [9]:
df_m = df.merge(df_geo, left_on='Postcode', right_on='Postal Code')

df_m.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1X,Scarborough,Upper Rouge,M1X,43.836125,-79.205636
1,M9W,Etobicoke,Northwest,M9W,43.706748,-79.594054
2,M3N,North York,Downsview Northwest,M3N,43.761631,-79.520999
3,M4B,East York,"Woodbine Gardens,Parkview Hill",M4B,43.706397,-79.309937
4,M1R,Scarborough,"Maryvale,Wexford",M1R,43.750072,-79.295849


When merging df and df_geo it contains Postcode and Postal Code. Only one is needed so Postal Code is dropped. 

In [10]:
df_m1 = df_m.drop('Postal Code', axis=1)

Taking a quick look at df_m1 after Postal Code has been dropped

In [11]:
df_m1.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
1,M9W,Etobicoke,Northwest,43.706748,-79.594054
2,M3N,North York,Downsview Northwest,43.761631,-79.520999
3,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
4,M1R,Scarborough,"Maryvale,Wexford",43.750072,-79.295849


Showing a map of Toronto

In [12]:
map_Toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10.7)
map_Toronto

Adding dots to show locations of different Postal Codes in toronto. Some postal codes can contain more than one Neighbourhood, and more than one postal code can belong to the same borough.

In [13]:
#neighborhoods = df_m1

for lat, lng, borough, neighborhood in zip(df_m1['Latitude'], df_m1['Longitude'], df_m1['Borough'], df_m1['Neighbourhood']):

    label = '{}, {}'.format(neighborhood, borough)

    label = folium.Popup(label, parse_html=True)

    folium.CircleMarker(

        [lat, lng],

        radius=5,

        popup=label,

        color='blue',

        fill=True,

        fill_color='#3186cc',

        fill_opacity=0.7,

        parse_html=False).add_to(map_Toronto)

Here is the previous map with postal code locations.

In [16]:
map_Toronto